In [ ]:
data_path = "/Users/ziyuwang/PycharmProjects/biosignal_anonym/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records500/00000/00001_hr.dat"

In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    df = data
    data = np.array([signal for signal, meta in data])
    return data, df

path = './ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

X, df = load_raw_data(Y, sampling_rate, path)

In [ ]:
Y

In [ ]:
df

In [ ]:
X

In [ ]:
len(df)

In [ ]:
signal, fields = wfdb.rdsamp('ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records500/00000/00001_hr')

In [ ]:
ecg_signal = signal[:, 0]  # Assuming the ECG signal is the first column
sample_rate = fields['fs']

In [ ]:
import wfdb
import neurokit2 as nk
import pandas as pd
cleaned_ecg = nk.ecg_clean(ecg_signal, sampling_rate=sample_rate)

# Find R-peaks
_, rpeaks = nk.ecg_peaks(cleaned_ecg, sampling_rate=sample_rate)

# Delineate the ECG signal
_, waves = nk.ecg_delineate(cleaned_ecg, rpeaks, sampling_rate=sample_rate, method="peak", show_type="peaks", show=True)

In [ ]:
rpeaks

In [ ]:
plot = nk.events_plot(rpeaks['ECG_R_Peaks'][:], cleaned_ecg[:], color='red')

In [ ]:
plot = nk.events_plot([waves['ECG_T_Peaks'], waves['ECG_P_Peaks'], waves['ECG_Q_Peaks'], waves['ECG_S_Peaks']], cleaned_ecg)

In [ ]:
plot = nk.events_plot(waves['ECG_P_Peaks'], cleaned_ecg, color='red')

In [ ]:
import math

def extract_pqrst_features(ecg_signal, sample_rate):
    cleaned_ecg = nk.ecg_clean(ecg_signal, sampling_rate=sample_rate)
    _, rpeaks = nk.ecg_peaks(cleaned_ecg, sampling_rate=sample_rate)

    # Delineate the ECG signal
    _, waves_peak = nk.ecg_delineate(cleaned_ecg, rpeaks, sampling_rate=sample_rate, method="peak")

    rp_amp_idx = [i for i in range(len(waves_peak['ECG_P_Peaks'])) if
                  not math.isnan(rpeaks['ECG_R_Peaks'][i]) and not math.isnan(waves_peak['ECG_P_Peaks'][i])]
    rq_amp_idx = [i for i in range(len(waves_peak['ECG_Q_Peaks'])) if
                  not math.isnan(rpeaks['ECG_R_Peaks'][i]) and not math.isnan(waves_peak['ECG_Q_Peaks'][i])]
    rs_amp_idx = [i for i in range(len(waves_peak['ECG_S_Peaks'])) if
                  not math.isnan(rpeaks['ECG_R_Peaks'][i]) and not math.isnan(waves_peak['ECG_S_Peaks'][i])]
    rt_amp_idx = [i for i in range(len(waves_peak['ECG_T_Peaks'])) if
                  not math.isnan(rpeaks['ECG_R_Peaks'][i]) and not math.isnan(waves_peak['ECG_T_Peaks'][i])]

    amp_p = [cleaned_ecg[waves_peak['ECG_P_Peaks'][i]] for i in rp_amp_idx]
    amp_rp = [cleaned_ecg[rpeaks['ECG_R_Peaks'][i]] for i in rp_amp_idx]

    amp_q = [cleaned_ecg[waves_peak['ECG_Q_Peaks'][i]] for i in rq_amp_idx]
    amp_rq = [cleaned_ecg[rpeaks['ECG_R_Peaks'][i]] for i in rq_amp_idx]

    amp_s = [cleaned_ecg[waves_peak['ECG_S_Peaks'][i]] for i in rs_amp_idx]
    amp_rs = [cleaned_ecg[rpeaks['ECG_R_Peaks'][i]] for i in rs_amp_idx]

    amp_t = [cleaned_ecg[waves_peak['ECG_T_Peaks'][i]] for i in rt_amp_idx]
    amp_rt = [cleaned_ecg[rpeaks['ECG_R_Peaks'][i]] for i in rt_amp_idx]

    rp_amp = np.array([a - b for a, b in zip(amp_rp, amp_p)])
    rq_amp = np.array([a - b for a, b in zip(amp_rq, amp_q)])
    rs_amp = np.array([a - b for a, b in zip(amp_rs, amp_s)])
    rt_amp = np.array([a - b for a, b in zip(amp_rt, amp_t)])

    pq_dis = np.array([a - b for a, b in zip(waves_peak['ECG_Q_Peaks'], waves_peak['ECG_P_Peaks'])])
    qr_dis = np.array([a - b for a, b in zip(rpeaks['ECG_R_Peaks'], waves_peak['ECG_Q_Peaks'])])
    rs_dis = np.array([a - b for a, b in zip(waves_peak['ECG_S_Peaks'], rpeaks['ECG_R_Peaks'])])
    st_dis = np.array([a - b for a, b in zip(waves_peak['ECG_T_Peaks'], waves_peak['ECG_S_Peaks'])])

    rp_avg = np.mean(rp_amp[~np.isnan(rp_amp)])
    rq_avg = np.mean(rq_amp[~np.isnan(rq_amp)])
    rs_avg = np.mean(rs_amp[~np.isnan(rs_amp)])
    rt_avg = np.mean(rt_amp[~np.isnan(rt_amp)])

    pq_avg = np.mean(pq_dis[~np.isnan(pq_dis)])
    qr_avg = np.mean(qr_dis[~np.isnan(qr_dis)])
    rs_avg = np.mean(rs_dis[~np.isnan(rs_dis)])
    st_avg = np.mean(st_dis[~np.isnan(st_dis)])

    return [pq_avg, qr_avg, rs_avg, st_avg, rp_avg, rq_avg, rs_avg, rt_avg]

In [ ]:
import os
import wfdb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def load_and_extract_features(metadata_df, directory_path, extract_features_func, max_users=1000):
    all_features = []
    all_labels = {'patient_id': [], 'age': [], 'gender': [], 'height': [], 'weight': [], 'age_binned': []}
    
    user_processed = 0
    age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # Define bins
    bin_labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']  # Labels for the bins
    
    failed_patients = []
    for index, row in metadata_df.iterrows():
        if user_processed >= max_users:
            break  # Limit the number of users processed
        
        try:
            record_path = os.path.join(directory_path, row['filename_lr'])
            print(f"Processing record {record_path} for patient ID {row['patient_id']}...")
            
            signal, fields = wfdb.rdsamp(record_path)
            lead_II_signal = signal[:, 1]  # Assuming Lead II is at index 1
            features = extract_features_func(lead_II_signal, fields['fs'])
            print(f"Extracted features: {features}")
            
            if features is not None:
                all_features.append(features)
                all_labels['patient_id'].append(row['patient_id'])
                all_labels['age'].append(row['age'])
                all_labels['gender'].append(row['sex'])
                all_labels['height'].append(row['height'])
                all_labels['weight'].append(row['weight'])

                # Perform age binning for the current patient and append
                current_age_bin = pd.cut([row['age']], bins=age_bins, labels=bin_labels, right=False)[0]
                all_labels['age_binned'].append(current_age_bin)
                
                user_processed += 1
        except Exception as e:
            print(f"Failed to process record {record_path}: {e}")
            failed_patients.append(row['patient_id'])
            
    
    print(f"Processed {user_processed} users.")
    return np.array(all_features), all_labels, failed_patients



def train_and_evaluate(X_train, y_train, model, task_name, is_classification=True, X_test=None, y_test=None):
    # Check if X_test and y_test are provided, otherwise split X_train and y_train
    if X_test is None or y_test is None:
        cutoff = int(len(X_train) * 0.8)
        X_train, X_test = X_train[:cutoff], X_train[cutoff:]
        y_train, y_test = y_train[:cutoff], y_train[cutoff:]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    if is_classification:
        print(f"Classification report for {task_name}:")
        print(classification_report(y_test, y_pred))
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        
        accuracy = accuracy_score(y_test, y_pred)
        # judge if it is a binary classification
        if len(np.unique(y_test)) == 2:
            precision = precision_score(y_test, y_pred, average='binary')
            recall = recall_score(y_test, y_pred, average='binary')
            f1 = f1_score(y_test, y_pred, average='binary')
        else:
            precision = precision_score(y_test, y_pred, average='weighted')  # Use 'binary' for binary classification, 'weighted' for multi-class
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')

        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1-score: {f1:.2f}")
    else:
        print(f"Mean Absolute Error for {task_name}: {mean_absolute_error(y_test, y_pred)}")
        
        print(f"R-squared for {task_name}: {r2_score(y_test, y_pred)}")



def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import BayesianRidge
from sklearn.naive_bayes import GaussianNB

# Define your models
models = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=500),
    'Baysesian': GaussianNB()
}

path_to_dataset = './ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
metadata_df = pd.read_csv(os.path.join(path_to_dataset, 'ptbxl_database.csv'))
metadata_df['filename_lr'] = metadata_df['filename_lr'].apply(lambda x: x.replace('.mat', ''))

max_users = 21837  # Make the number of users adjustable
train_ratio = 0.8  # Adjustable train-test split ratio

In [ ]:
# Gender prediction with patient ID split

# Clean the DataFrame based on 'sex' column and limit the data to 'max_users'
clean_metadata_df_gender = metadata_df.dropna(subset=['sex']).head(max_users)

# Load and extract features
features_gender, labels_gender, failed_patients = load_and_extract_features(clean_metadata_df_gender, path_to_dataset, extract_pqrst_features, max_users)

# Report on cleaned data
original_count = len(metadata_df)
clean_count = len(clean_metadata_df_gender) - len(failed_patients)  # Adjusted for failed patients
print(f"Original user count: {original_count}")
print(f"Clean user count (no NaNs in sex, excluding failed patients): {clean_count}")
print(f"Ratio of clean data: {clean_count / original_count:.2f}")

# Encode gender labels
le = LabelEncoder()
y_gender = le.fit_transform(labels_gender['gender'])

# Scale features
scaler = StandardScaler()
X_scaled_gender = scaler.fit_transform(features_gender)

unique_patient_ids = clean_metadata_df_gender['patient_id'].unique()
split_index = int(len(unique_patient_ids) * 0.8)

train_patient_ids = unique_patient_ids[:split_index]
test_patient_ids = unique_patient_ids[split_index:]

# Distinguished by patient_id: Perform an 80-20 split based on unique patient IDs
train_patient_ids = set(train_patient_ids) - set(failed_patients)  # Exclude failed patients
test_patient_ids = set(test_patient_ids) - set(failed_patients)  # Exclude failed patients

# Filter the dataset for training and testing, excluding failed patients
train_mask = np.isin(labels_gender['patient_id'], list(train_patient_ids))
test_mask = np.isin(labels_gender['patient_id'], list(test_patient_ids))

X_train_gender = X_scaled_gender[train_mask]
y_train_gender = y_gender[train_mask]

X_test_gender = X_scaled_gender[test_mask]
y_test_gender = y_gender[test_mask]

print("Predicting Gender:")
for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_gender, y_train_gender)
    y_pred = model.predict(X_test_gender)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_gender, y_train_gender, model, 'Gender', is_classification=True, X_test=X_test_gender, y_test=y_test_gender)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
print("Predicting Gender:")
# Fit the imputer on the training data and transform it
X_train_gender_imputed = imputer.fit_transform(X_train_gender)

# Transform the test data using the same imputer
X_test_gender_imputed = imputer.transform(X_test_gender)

for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_gender_imputed, y_train_gender)
    y_pred = model.predict(X_test_gender_imputed)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_gender_imputed, y_train_gender, model, 'Gender', is_classification=True, X_test=X_test_gender_imputed, y_test=y_test_gender)

In [ ]:
# Age prediction with patient ID split

max_users = 21837  # Make the number of users adjustable

# Clean the DataFrame based on 'age' column and limit the data to 'max_users'
clean_metadata_df_age = metadata_df.dropna(subset=['age']).head(max_users)

# Load and extract features, adjusting the function to handle age
features_age, labels_age, failed_patients_age = load_and_extract_features(clean_metadata_df_age, path_to_dataset, extract_pqrst_features, max_users)

# Convert age to categorical bins for classification
# age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

age_bins = [0, 20, 40, 60, 80, 100]
labels_age['age_binned'] = pd.cut(labels_age['age'], bins=age_bins, labels=False)  # Using integer labels for bins

# Encode binned age labels
le_age = LabelEncoder()
y_age_binned = le_age.fit_transform(labels_age['age_binned'])

# Scale features
scaler_age = StandardScaler()
X_scaled_age = scaler_age.fit_transform(features_age)

# Perform a patient ID split, adjusted for age analysis
train_patient_ids_age = set(clean_metadata_df_age['patient_id'][:split_index].unique()) - set(failed_patients_age)
test_patient_ids_age = set(clean_metadata_df_age['patient_id'][split_index:].unique()) - set(failed_patients_age)

train_mask_age = np.isin(labels_age['patient_id'], list(train_patient_ids_age))
test_mask_age = np.isin(labels_age['patient_id'], list(test_patient_ids_age))

X_train_age_binned = X_scaled_age[train_mask_age]
y_train_age_binned = y_age_binned[train_mask_age]

X_test_age_binned = X_scaled_age[test_mask_age]
y_test_age_binned = y_age_binned[test_mask_age]

print("Predicting Age Bin:")
for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_age_binned, y_train_age_binned)
    y_pred = model.predict(X_test_age_binned)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_age_binned, y_train_age_binned, model, 'Age Bin', is_classification=True, X_test=X_test_age_binned, y_test=y_test_age_binned)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
print("Predicting Age Bin:")
# Fit the imputer on the training data and transform it

X_train_age_binned = X_scaled_age[train_mask_age]
y_train_age_binned = y_age_binned[train_mask_age]

X_test_age_binned = X_scaled_age[test_mask_age]
y_test_age_binned = y_age_binned[test_mask_age]

X_train_age_binned_imputed = imputer.fit_transform(X_train_age_binned)

# Transform the test data using the same imputer
X_test_age_binned_imputed = imputer.transform(X_test_age_binned)

for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_age_binned_imputed, y_train_age_binned)
    y_pred = model.predict(X_test_age_binned_imputed)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_age_binned_imputed, y_train_age_binned, model, 'Age Bin', is_classification=True, X_test=X_test_age_binned_imputed, y_test=y_test_age_binned)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming metadata_df, path_to_dataset, and extract_pqrst_features are predefined
# Adjust max_users to control the number of patients processed
max_users = 21837

# Load and extract features for patient reidentification
features, labels, failed_patients = load_and_extract_features(metadata_df, path_to_dataset, extract_pqrst_features, max_users)

# Assuming 'patient_id' is included in labels and corresponds to each row in features
patient_ids = np.array(labels['patient_id'])

# Remove data related to failed patients
success_mask = ~np.isin(patient_ids, failed_patients)
features_clean = features[success_mask]
patient_ids_clean = patient_ids[success_mask]

# Encode patient IDs as numeric labels
le = LabelEncoder()
y_encoded = le.fit_transform(patient_ids_clean)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_clean)

# Splitting data: Instead of a random train-test split, do a stratified split based on patient IDs
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, shuffle=False)


for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train, y_train, model, 'Patient ID', is_classification=True, X_test=X_test, y_test=y_test)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
print("Predicting ID:")
# Fit the imputer on the training data and transform it
X_train_imputed = imputer.fit_transform(X_train)

# Transform the test data using the same imputer
X_test_imputed = imputer.transform(X_test)

for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_imputed, y_train)
    y_pred = model.predict(X_test_imputed)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_imputed, y_train, model, 'Patient ID', is_classification=True, X_test=X_test_imputed, y_test=y_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming metadata_df, path_to_dataset, and extract_pqrst_features are predefined
# Adjust max_users to control the number of patients processed
max_users = 100

# Load and extract features for patient reidentification
features, labels, failed_patients = load_and_extract_features(metadata_df, path_to_dataset, extract_pqrst_features, max_users)


# Assuming 'patient_id' is included in labels and corresponds to each row in features
patient_ids = np.array(labels['patient_id'])

# Remove data related to failed patients
success_mask = ~np.isin(patient_ids, failed_patients)
features_clean = features[success_mask]
patient_ids_clean = patient_ids[success_mask]

# Encode patient IDs as numeric labels
le = LabelEncoder()
y_encoded = le.fit_transform(patient_ids_clean)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_clean)

# Splitting data: Instead of a random train-test split, do a stratified split based on patient IDs
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, shuffle=False)



In [ ]:
features_clean

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
print("Predicting ID:")
# Fit the imputer on the training data and transform it
X_train_imputed = imputer.fit_transform(X_train)

# Transform the test data using the same imputer
X_test_imputed = imputer.transform(X_test)

print(y_test)

for model_name, model in models.items():
    print(f"\nUsing model: {model_name}")
    model.fit(X_train_imputed, y_train)
    y_pred = model.predict(X_test_imputed)
    # Assuming train_and_evaluate is defined and properly handles evaluation
    train_and_evaluate(X_train_imputed, y_train, model, 'Patient ID', is_classification=True, X_test=X_test_imputed, y_test=y_test)